In [1]:
%config Completer.use_jedi = False
import warnings

warnings.filterwarnings(action='ignore')

# 의사 결정 트리(Decision Tree)
의사 결정 트리는 이름처럼 의사 결정을 하는데 나무(tree)를 사용한다는 뜻이다.
의사 결정 트리에서는 테스트를 위쪽에 놓고 해당 테스트로 발생 가능한 결과를 아래쪽에 놓는다. 의사 결정 트리의 결과는 또 다른 테스트가 될 수 있고 즉, 상위 테스트로 판별하지 못한 데이터는 또 다른 테스트인 자식 테스트를 사용해 데이터 분류가 가능하다.

※ 엔트로피(Entropy)<br/>
테스트 성능 평가를 위해 엔트로피라는 개념을 사용한다.
엔트로피는 불순도(impurity) 정도를 측정하며, 낮을수록 좋다. 이때, 불순도난 노드에 서로 다른 데이터가 얼마나 섰여있는지를 의미한다. 불순도가 낮을수록 데이터가 섞여있지 않다는 것을 의미한다. 엔트로피는 아래와 같은 식을 따른다.
$$entropy(d) = -\sum P(x)logP(x) = -\sum_{i=1}^k p(i|d)log_2(p(i|d))$$
위 식에서 $p(i|d)$는 노드 $d$가 주어질 때 $i$ 클래스에 속할 확률이다. 이진 분류 문제는 $k = 2$이며, 이를 적용하면 아래와 같이 표현할 수 있다.
$$entropy(d) = -\frac {P}{T} log_2 \frac {P}{T} - \frac {N}{T} log_2 \frac {N}{T}$$
위 공식에서 $T$는 노드 속 전체(total) 데이터 수, $P$는 Positive 데이터 수, $N$은 Negative 데이터 수를 의미한다.

※ 지니 계수(Gini Index)<br/>
지니 계수는 엔트로피와 함께 불순도를 측정하는 또 다른 방법이다.
지니 계수는 데이터셋에서 랜덤으로 선택한 데이터에 임의로 라벨을 정했을 때 틀릴 확률을 의미한다. 만일 해당 데이터셋의 데이터가 모두 동일하게 라벨링되어 있다면 지니 계수는 0을 갖게 되며 이는 불순도가 0임을 의미한다.

의사 결정 트리 알고리즘을 활용해 와인 종류를 구분한다.

In [2]:
# 데이터 불러오기
from sklearn import datasets  # 와인 데이터를 사용하기 위해 import 한다.

raw_wine = datasets.load_wine()  # 와인 데이터를 불러온다.

In [3]:
# 피쳐/타겟 데이터 지정
X = raw_wine.data  # 와인 피쳐 데이터를 저장한다.
y = raw_wine.target  # 와인 타겟 데이터를 저장한다.

In [4]:
# 트레이닝/테스트 데이터 분할
from sklearn.model_selection import train_test_split  # 트레이닝/테스트 데이터 분할을 위해 import 한다.

X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)  # 트레이닝 데이터와 테스트 데이터로 분할한다.
X_tn.shape, X_te.shape

((133, 13), (45, 13))

In [5]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler  # 데이터 표준화를 위해 import 한다.

std_scale = StandardScaler()  # 표준화 스케일러 객체를 만든다.
# 표준화는 트레이닝 데이터를 기반으로 실행하므로 트레이닝 피쳐 데이터 X_tn을 표준화 스케일러에 적합시킨다.
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)  # 트레이닝 피쳐 데이터 X_tn을 표준화 한다.
X_te_std = std_scale.transform(X_te)  # 테스트 피쳐 데이터 X_te를 표준화 한다.

In [6]:
# 데이터 학습
# 분류 문제는 DecisionTreeClassifier를 사용하고 회귀 문제는 DecisionTreeRegressor를 사용한다.
from sklearn.tree import DecisionTreeClassifier  # 의사 결정 트리 알고리즘을 사용하기 위해 import 한다.

clf_tree = DecisionTreeClassifier(random_state=0)  # 의사 결정 트리 모델 객체를 만든다.
# 표준화된 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn을 넣어서 선형 회귀 분석 알고리즘을 학습시킨다.
clf_tree.fit(X_tn_std, y_tn)

DecisionTreeClassifier(random_state=0)

In [7]:
# 데이터 예측
pred_tree = clf_tree.predict(X_te_std)  # 표준화된 테스트 데이터 X_te_std로 예측한다.
pred_tree

array([0, 2, 1, 0, 1, 1, 0, 2, 1, 1, 2, 2, 0, 1, 2, 1, 0, 0, 2, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 2, 0, 0, 1, 0, 0, 0, 2, 1, 1, 2, 1, 0, 1, 1,
       1])

In [8]:
# f1 score 평가
from sklearn.metrics import f1_score  # f1 score를 평가하기 위해 import 한다.

# f1_score() 함수의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_tree)를 넘겨 f1 score를 평가한다.
# average 옵션으로 f1 score 계산 방법을 지정한다. 기본값은 binary이고 예측 결과가 2가지 일 때 사용한다.
# multi-class 인데 average 옵션을 지정하지 않으면 기본값 binary로 지정되서 에러가 발생된다.
# None: 각 클래스별 f1 score를 리턴한다.
# micro: 전체 클래스에 대해 각 클래스에 대한 TP, FP, FN을 합한 뒤에 f1 score를 계산한다.
# macro: 각 클래스에 대해 f1 score를 계산한 뒤 산술 평균을 계산해서 f1 score를 계산한다.
# weighted: 각 클래스에 대해 f1 score를 게산한 뒤 각 클래스별 데이터 비율에 따른 가중 평균으로 f1 score를 계산한다.
f1 = f1_score(y_te, pred_tree, average='macro')
f1

0.9349141206870346

In [9]:
# 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix  # 혼돈 행렬을 만들기 위해 import 한다.

# confusion_matrix() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 혼돈 행렬을 만든다.
conf_matrix = confusion_matrix(y_te, pred_tree)
conf_matrix

array([[14,  2,  0],
       [ 0, 20,  1],
       [ 0,  0,  8]])

In [10]:
# 분류 리포트 확인
from sklearn.metrics import classification_report  # 분류 리포트를 출력하기 위해 import 한다.

# classification_report() 함수에 실제 타겟 데이터와 예측된 데이터를 넘겨 분류 리포트를 만든다.
class_report = classification_report(y_te, pred_tree)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      0.88      0.93        16
           1       0.91      0.95      0.93        21
           2       0.89      1.00      0.94         8

    accuracy                           0.93        45
   macro avg       0.93      0.94      0.93        45
weighted avg       0.94      0.93      0.93        45

